<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.1
    Uninstalling protobuf-6.33.1:
      Successfully uninstalled protobuf-6.33.1


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-11-25 15:23:00
-------------------
qualified stocks: 87
with latest results: 25
still star stocks: 15
-------------------
Initial Investment:  1.26 C
CY Investment:  1.53 C
Reserve:  400.00
Current:  1.42 C
-------------------
Today PnL: -26.03 K (-0.18%)
Current PnL: -23.59 L (-15.46%)
CY Booked + Current PnL: -9.70 L (-6.36%)
-------------------
Total profit:  1.28 L
Total loss:  -24.87 L
-------------------
Total Booked + Current PnL: 17.28 L (13.75%)
Total Booked PnL: 40.87 L (32.53%)
Curr Year Booked PnL: 13.89 L (9.77%)
Prev Year Booked PnL: 26.98 L (21.48%)
Est FTT:  2.31 C
Est FTT PnL: 89.00 L (62.59%)
Deployed:  1.26 C
Current:  1.42 C
CAGR/XIRR %: 7.05%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT','Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,TTKPRESTIG,770.00,0.02,-14.21,16.67,0.09,14413.0,-14317.0,86460.0,100.64,49.0,M-SC,3.97,253.0,-0.99,0.61,13.59,OX40N,NTT,DURABLES
39,INDIGOPNTS,1408.00,0.06,-9.28,10.25,0.02,16232.0,-16199.0,158360.0,141.63,70.0,M-SC,6.67,234.0,-1.00,1.11,37.75,OX40N,NTT,PAINTS
35,ICICIGI,2252.93,-0.01,6.56,12.79,20.19,23758.0,11438.0,185758.0,-16.49,50.0,X-MC,7.57,68.0,0.48,1.31,22.67,X40,ATH,INSURANCE
2,ABBOTINDIA,35195.00,1.95,0.95,16.33,17.44,24703.0,1428.0,151275.0,-13.10,61.0,X-MC,4.19,63.0,0.06,1.06,20.29,X40,ATH,PHARMA
52,MEDANTA,1486.00,0.02,4.43,18.81,24.08,24937.0,5624.0,132574.0,-4.04,48.0,X-SC,4.91,89.0,0.23,0.93,24.92,XY24,NTT,HEALTHCARE


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
62,ROUTE,2227.21,3.82,-49.41,234.17,69.05,157634.0,-65750.0,67316.0,-58.98,37.0,H-SC,23.55,140.0,-0.42,0.47,3.82,SR,ATH,IT
56,QUESS,424.00,3.62,-26.56,94.51,42.84,45117.0,-17268.0,47738.0,-52.07,44.0,X-SC,37.31,83.0,-0.38,0.34,3.87,XY24,NTT,MISC
61,REPCOHOME,880.00,3.14,-10.27,111.54,89.82,286756.0,-29417.0,257088.0,-55.29,51.0,H-SC,10.45,160.0,-0.10,1.81,36.02,XY24,NTT,FINANCE
58,RAJOOENG,143.10,2.61,-28.98,96.57,39.61,70303.0,-29700.0,72800.0,-52.00,26.0,H-SC,20.02,135.0,-0.42,0.51,2.61,AR,ATH,MISC
30,HATHWAY,31.40,2.35,-27.98,140.61,73.29,110098.0,-30420.0,78300.0,809.42,38.0,H-SC,4.60,172.0,-0.28,0.55,4.74,XR,NTT,ENTERTAINMENT


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
34,HONAUT,58357.33,-3.72,-14.25,67.14,43.32,93768.0,-23212.0,139660.0,-26.34,43.0,X-SC,2.02,90.0,-0.25,0.98,7.23,X40N,ATH,ELECTRICAL
3,ACC,3906.00,-2.38,-21.49,109.09,64.15,203791.0,-51141.0,186810.0,-54.13,53.0,X-SC,1.64,82.0,-0.25,1.31,4.79,XY24,BTT,CEMENT
57,RAJESHEXPO,518.00,-2.04,-63.77,176.24,0.07,87579.0,-87484.0,49693.0,1720.71,54.0,L-SC,1.52,268.0,-1.00,0.35,23.59,OX40N,NTT,JEWELLERY
12,BERGEPAINT,680.00,-1.94,1.20,19.71,21.14,45345.0,2721.0,230060.0,-5.97,55.0,X-MC,3.82,74.0,0.06,1.62,29.67,XY24,NTT,PAINTS
22,DIXON,18931.72,-1.85,-7.53,31.49,21.58,40805.0,-10557.0,129582.0,-53.35,28.0,X-MC,6.89,56.0,-0.26,0.91,14.72,X40N,ATH,IT


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
85,WIPRO,420.0,-0.47,0.95,70.65,72.27,120849.0,1605.0,171053.0,-13.34,54.0,M-LC,4.54,99.0,0.01,1.2,7.18,XR,NTT,IT


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,0.69,-4.63,118.00,107.91,166157.0,-6833.0,140811.0,-22.58,36.0,M-SC,10.45,216.0,-0.04,0.99,0.69,OX40N,ATH,CABLES
39,INDIGOPNTS,1408.00,0.06,-9.28,10.25,0.02,16232.0,-16199.0,158360.0,141.63,70.0,M-SC,6.67,234.0,-1.00,1.11,37.75,OX40N,NTT,PAINTS
78,TTKPRESTIG,770.00,0.02,-14.21,16.67,0.09,14413.0,-14317.0,86460.0,100.64,49.0,M-SC,3.97,253.0,-0.99,0.61,13.59,OX40N,NTT,DURABLES
47,KANSAINER,340.00,-0.44,-22.91,47.20,13.47,98120.0,-61785.0,207882.0,-68.31,26.0,H-SC,1.39,158.0,-0.63,1.46,5.70,XY24,NTT,PAINTS
44,JCHAC,2282.00,-0.37,-23.49,30.68,-0.01,30001.0,-30018.0,97787.0,19921.74,54.0,M-SC,1.67,233.0,-1.00,0.69,10.73,OX40N,NTT,AC


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
85,WIPRO,420.00,-0.47,0.95,70.65,72.27,120849.0,1605.0,171053.0,-13.34,54.0,M-LC,4.54,99.0,0.01,1.20,7.18,XR,NTT,IT
38,INDIAMART,4810.62,2.22,1.36,103.94,106.72,129942.0,1680.0,125016.0,-49.87,41.0,H-SC,3.76,139.0,0.01,0.88,25.06,AR,ATH,MISC


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
38,INDIAMART,4810.62,2.22,1.36,103.94,106.72,129942.0,1680.0,125016.0,-49.87,41.0,H-SC,3.76,139.0,0.01,0.88,25.06,AR,ATH,MISC
85,WIPRO,420.00,-0.47,0.95,70.65,72.27,120849.0,1605.0,171053.0,-13.34,54.0,M-LC,4.54,99.0,0.01,1.20,7.18,XR,NTT,IT
25,FINCABLES,1641.55,0.69,-4.63,118.00,107.91,166157.0,-6833.0,140811.0,-22.58,36.0,M-SC,10.45,216.0,-0.04,0.99,0.69,OX40N,ATH,CABLES
84,WHIRLPOOL,2270.00,1.05,-4.19,89.88,81.92,98855.0,-4812.0,109986.0,-43.90,36.0,M-SC,4.57,236.0,-0.05,0.77,30.21,XR,NTT,DURABLES
86,ZYDUSLIFE,1286.17,0.12,-2.16,38.63,35.63,79208.0,-4530.0,205044.0,-15.85,34.0,H-MC,4.38,119.0,-0.06,1.44,14.15,AR,ATH,PHARMA


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
76,TMPV,600.00,-1.60,-16.46,70.19,42.18,162083.0,-45490.0,230920.0,-25.52,20.0,X-LC,0.64,3.0,-0.28,1.62,0.00,XY24,NTT,AUTO
46,JSWINFRA,342.00,-0.74,-10.04,28.09,15.23,50625.0,-20122.0,180225.0,-22.98,23.0,X-MC,6.67,66.0,-0.40,1.27,18.97,X40N,NTT,REALTY
60,RELAXO,1176.00,-0.86,-48.27,194.40,52.31,145989.0,-70063.0,75097.0,-44.92,25.0,X-SC,3.99,91.0,-0.48,0.53,0.52,X40N,NTT,FOOTWEAR
11,BATAINDIA,2096.00,0.05,-35.09,108.31,35.20,90455.0,-45155.0,83515.0,9.46,25.0,X-SC,12.86,92.0,-0.50,0.59,0.58,X40,NTT,FOOTWEAR
22,DIXON,18931.72,-1.85,-7.53,31.49,21.58,40805.0,-10557.0,129582.0,-53.35,28.0,X-MC,6.89,56.0,-0.26,0.91,14.72,X40N,ATH,IT


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
76,TMPV,600.0,-1.60,-16.46,70.19,42.18,162083.0,-45490.0,230920.0,-25.52,20.0,X-LC,0.64,3.0,-0.28,1.62,0.00,XY24,NTT,AUTO
43,ITC,452.0,-0.74,-2.13,12.84,10.44,25150.0,-4269.0,195869.0,-41.56,39.0,X-LC,0.84,5.0,-0.17,1.38,3.31,X40,NTT,FMCG
36,ICICIPRULI,790.0,0.84,1.53,28.99,30.97,52022.0,2710.0,179448.0,-22.41,50.0,X-MC,1.58,75.0,0.05,1.26,14.29,X40,ATH,INSURANCE
3,ACC,3906.0,-2.38,-21.49,109.09,64.15,203791.0,-51141.0,186810.0,-54.13,53.0,X-SC,1.64,82.0,-0.25,1.31,4.79,XY24,BTT,CEMENT
66,SIEMENS,4671.5,0.53,-14.54,46.87,25.51,74537.0,-27065.0,159030.0,-2.45,55.0,H-LC,1.88,49.0,-0.36,1.12,17.19,AR,ATH,ELECTRICAL


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
9,BAJAJHFL,181.50,-0.78,-14.17,72.96,48.45,125566.0,-28404.0,172102.0,-20.30,28.0,X-MC,6.84,64.0,-0.23,1.21,0.00,X40N,ATH,FINANCE
54,PAGEIND,51605.07,-0.84,-5.86,33.92,26.07,52284.0,-9600.0,154140.0,-30.68,33.0,X-MC,9.05,55.0,-0.18,1.08,0.00,X40,ATH,APPARELS
76,TMPV,600.00,-1.60,-16.46,70.19,42.18,162083.0,-45490.0,230920.0,-25.52,20.0,X-LC,0.64,3.0,-0.28,1.62,0.00,XY24,NTT,AUTO
60,RELAXO,1176.00,-0.86,-48.27,194.40,52.31,145989.0,-70063.0,75097.0,-44.92,25.0,X-SC,3.99,91.0,-0.48,0.53,0.52,X40N,NTT,FOOTWEAR
11,BATAINDIA,2096.00,0.05,-35.09,108.31,35.20,90455.0,-45155.0,83515.0,9.46,25.0,X-SC,12.86,92.0,-0.50,0.59,0.58,X40,NTT,FOOTWEAR


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
56,QUESS,424.00,3.62,-26.56,94.51,42.84,45117.0,-17268.0,47738.0,-52.07,44.0,X-SC,37.31,83.0,-0.38,0.34,3.87,XY24,NTT,MISC
60,RELAXO,1176.00,-0.86,-48.27,194.40,52.31,145989.0,-70063.0,75097.0,-44.92,25.0,X-SC,3.99,91.0,-0.48,0.53,0.52,X40N,NTT,FOOTWEAR
11,BATAINDIA,2096.00,0.05,-35.09,108.31,35.20,90455.0,-45155.0,83515.0,9.46,25.0,X-SC,12.86,92.0,-0.50,0.59,0.58,X40,NTT,FOOTWEAR
22,DIXON,18931.72,-1.85,-7.53,31.49,21.58,40805.0,-10557.0,129582.0,-53.35,28.0,X-MC,6.89,56.0,-0.26,0.91,14.72,X40N,ATH,IT
52,MEDANTA,1486.00,0.02,4.43,18.81,24.08,24937.0,5624.0,132574.0,-4.04,48.0,X-SC,4.91,89.0,0.23,0.93,24.92,XY24,NTT,HEALTHCARE


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4389.97,-0.64,-15.20,40.65,19.28,119260.0,-52573.0,293383.0,-26.21,58.0,X-LC,7.53,1.0,-0.44,2.06,8.46,X40,ATH,IT
41,INFY,2275.00,-1.16,4.16,48.69,54.87,158676.0,13002.0,325890.0,-18.88,56.0,X-LC,3.01,2.0,0.08,2.29,12.78,X40,BTT,IT
76,TMPV,600.00,-1.60,-16.46,70.19,42.18,162083.0,-45490.0,230920.0,-25.52,20.0,X-LC,0.64,3.0,-0.28,1.62,0.00,XY24,NTT,AUTO
82,VBL,671.64,0.67,-9.28,49.55,35.67,141974.0,-29316.0,286526.0,-19.99,43.0,X-LC,5.95,4.0,-0.21,2.02,3.29,X40N,ATH,FMCG
43,ITC,452.00,-0.74,-2.13,12.84,10.44,25150.0,-4269.0,195869.0,-41.56,39.0,X-LC,0.84,5.0,-0.17,1.38,3.31,X40,NTT,FMCG


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
6,ASIANTILES,137.00,0.29,-12.98,108.78,81.67,88798.0,-12179.0,81631.0,7338.89,51.0,L-SC,18.75,271.0,-0.14,0.57,58.08,XR,NTT,CERAMICS
68,SONACOMS,806.63,-0.85,-13.24,60.84,39.54,53394.0,-13397.0,87762.0,-32.10,63.0,M-SC,6.70,220.0,-0.25,0.62,23.75,AR,ATH,AUTO
51,MASFIN,398.61,-0.81,-5.97,29.80,22.05,27455.0,-5850.0,92130.0,-19.15,47.0,H-SC,7.29,164.0,-0.21,0.65,33.92,XR,ATH,FINANCE
70,SURYODAY,216.00,0.28,-19.61,53.04,23.03,76358.0,-35108.0,143963.0,57.89,48.0,H-SC,10.61,167.0,-0.46,1.01,42.84,XR,NTT,BANKS
39,INDIGOPNTS,1408.00,0.06,-9.28,10.25,0.02,16232.0,-16199.0,158360.0,141.63,70.0,M-SC,6.67,234.0,-1.00,1.11,37.75,OX40N,NTT,PAINTS


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
39,INDIGOPNTS,1408.00,0.06,-9.28,10.25,0.02,16232.0,-16199.0,158360.0,141.63,70.0,M-SC,6.67,234.0,-1.00,1.11,37.75,OX40N,NTT,PAINTS
5,ASIANPAINT,3460.25,-0.17,0.46,20.39,20.95,51572.0,1162.0,252930.0,-0.49,75.0,X-LC,13.37,31.0,0.02,1.78,36.26,X40,ATH,PAINTS
40,INDUSINDBK,1800.00,0.47,-37.98,114.29,32.89,54722.0,-29326.0,47880.0,-691.63,60.0,L-MC,5.58,259.0,-0.54,0.34,31.86,XR,NTT,BANKS
79,UNITDSPR,1644.00,-0.15,4.97,15.05,20.77,36991.0,11644.0,245788.0,-4.14,58.0,X-MC,3.56,62.0,0.31,1.73,12.24,X40N,NTT,BREWERIES
12,BERGEPAINT,680.00,-1.94,1.20,19.71,21.14,45345.0,2721.0,230060.0,-5.97,55.0,X-MC,3.82,74.0,0.06,1.62,29.67,XY24,NTT,PAINTS


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.76
1,25,44.60
2,50,75.43


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    45.68
MC    29.23
LC    25.05
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     27.94
X40      21.20
X40N     11.86
XR        9.62
XY25      9.05
AR        9.00
OX40N     7.67
X200      1.85
SR        0.99
MH        0.78
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    23.97
X-MC    22.83
X-LC    19.92
M-SC    12.04
X-SC     8.26
H-MC     4.70
H-LC     2.97
L-SC     1.41
M-MC     1.36
M-LC     1.20
L-LC     0.96
L-MC     0.34
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,16.80,-5.31,39.37
IT,13.68,-16.14,76.10
FINANCE,9.43,-15.66,64.53
MISC,7.42,-23.08,73.50
PAINTS,5.97,-8.73,24.88
ELECTRICAL,5.91,-11.75,52.68
INSURANCE,4.47,0.49,34.58
PHARMA,3.90,-0.54,32.72
AUTO,3.31,-21.95,74.24


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3060558.0,21
AR,1293570.0,10
XR,1293210.0,13
X40,1022593.0,14
X40N,762806.0,9
OX40N,700202.0,10
XY25,353417.0,6
SR,279427.0,2
MH,72991.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3567961.0,25
M-SC,1345979.0,15
X-MC,1233993.0,16
X-LC,930829.0,11
X-SC,744979.0,8
H-MC,410100.0,3
L-SC,260569.0,3
H-LC,135927.0,2
M-LC,120849.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1215156.0      6
           AR         877504.0      5
           XR         796072.0      7
M-SC       XY24       787316.0      6
X-MC       X40        443175.0      7
X-LC       X40        405033.0      5
X-MC       XY24       355094.0      3
H-SC       OX40N      326811.0      4
X-SC       X40N       296749.0      3
M-SC       OX40N      285812.0      5
H-SC       SR         279427.0      2
X-SC       XY24       273845.0      3
X-MC       X40N       253987.0      4
X-LC       XY24       236302.0      2
H-MC       AR         217255.0      2
X-LC       X40N       212070.0      2
H-MC       XY24       192845.0      1
X-MC       XY25       181737.0      2
X-SC       X40        174385.0      2
L-SC       XR         172990.0      2
M-SC       XR         148577.0      2
           AR         124274.0      2
M-LC       XR         120849.0      1
L-SC       OX40N       87579.0      1
X-LC       XY25        77424.0      2
H-LC       AR          74537.0      1
H-SC       MH          72991.0      1
H-LC       X200        61390.0      1
L-MC       XR          54722.0      1
M-MC       XY25        51754.0      1
L-LC       XY25        42502.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 25.0 seconds
